# 🤖 Análise de Consenso entre LLMs - Anotação sem LLM Hacking

## 1) Setup e Configuração

In [1]:
import sys
from loguru import logger
import pandas as pd

logger.remove()
logger.add(
    sys.stdout,
    format="<green>{time:HH:mm:ss}</green> | <level>{level: <8}</level> | <level>{message}</level>",
    level="INFO"
)

logger.success("✓ Setup completo")

01:32:22 | SUCCESS  | ✓ Setup completo


## 2) Carregar Dataset

In [2]:
from src.utils.data_loader import load_hf_dataset, load_hf_dataset_as_dataframe, list_available_datasets, get_dataset_info

logger.info("Datasets disponíveis:")
for dataset in list_available_datasets():
    logger.info(f"  - {dataset}")

01:32:23 | Datasets disponíveis:
01:32:23 |   - agnews
01:32:23 |   - mpqa
01:32:23 |   - webkb
01:32:23 |   - ohsumed
01:32:23 |   - acm
01:32:23 |   - yelp_2013
01:32:23 |   - dblp
01:32:23 |   - books
01:32:23 |   - reut90
01:32:23 |   - wos11967
01:32:23 |   - twitter
01:32:23 |   - trec
01:32:23 |   - wos5736
01:32:23 |   - sst1
01:32:23 |   - pang_movie
01:32:23 |   - movie_review
01:32:23 |   - vader_movie
01:32:23 |   - subj
01:32:23 |   - sst2
01:32:23 |   - yelp_reviews
01:32:23 |   - 20ng
01:32:23 |   - medline


In [3]:
dataset_name = "sst1"  # Ajuste conforme necessário

texts, categories, ground_truth = load_hf_dataset(dataset_name)

logger.info(f"Textos: {len(texts)}")
logger.info(f"Categorias: {categories}")
logger.info(f"Ground truth: {'Sim' if ground_truth else 'Não'}")

01:32:23 | Carregando dataset: sst1
01:32:23 | Combinando splits: ['train', 'test']
01:32:32 |   ✓ train: 106695 exemplos
01:32:38 |   ✓ test: 11855 exemplos
01:32:38 | Total combinado: 118550 exemplos
01:32:39 | Categorias extraídas automaticamente: [0, 1, 2, 3, 4]
01:32:39 | Amostra reduzida para 500 exemplos (seed=42)
01:32:39 | Coluna de texto: text
01:32:39 | Ground truth carregado da coluna 'label'
01:32:39 | Textos: 500
01:32:39 | Categorias: [0, 1, 2, 3, 4]
01:32:39 | Ground truth: Sim


In [4]:
# Visualizar amostra
logger.info("Amostra dos textos:")
for i, text in enumerate(texts[:3]):
    logger.info(f"{i+1}. {text[:100]}...")
    if ground_truth:
        logger.info(f"   Label: {ground_truth[i]}")

01:32:39 | Amostra dos textos:
01:32:39 | 1. impostor has a handful of thrilling moments and a couple of good performances , but the movie does n...
01:32:39 |    Label: 1
01:32:39 | 2. the acting in pauline and paulette is good all round , but what really sets the film apart is debrau...
01:32:39 |    Label: 4
01:32:39 | 3. mr. polanski is in his element here : alone , abandoned , but still consoled by his art , which is m...
01:32:39 |    Label: 3


In [5]:
df, categories = load_hf_dataset_as_dataframe(dataset_name)

01:32:39 | Carregando dataset: sst1
01:32:39 | Combinando splits: ['train', 'test']
01:32:45 |   ✓ train: 106695 exemplos
01:32:52 |   ✓ test: 11855 exemplos
01:32:52 | Total combinado: 118550 exemplos
01:32:52 | Categorias extraídas automaticamente: [0, 1, 2, 3, 4]
01:32:52 | Amostra reduzida para 500 exemplos (seed=42)
01:32:52 | Coluna de texto: text
01:32:52 | Ground truth carregado da coluna 'label'
01:32:52 | DataFrame criado com 500 linhas


## 3) Configurar Modelos LLM

### Opção A: Usar apenas parâmetros padrão (temp=0)

In [6]:
from src.llm_annotation_system.annotation.llm_annotator import LLMAnnotator
from src.experiments.base_experiment import DEFAULT_MODELS, BASE_ANNOTATION_PROMPT

annotator = LLMAnnotator(
    dataset_name=dataset_name,
    categories=categories,
    models=DEFAULT_MODELS,
    prompt_template=BASE_ANNOTATION_PROMPT,
    use_langchain_cache=True,
    use_alternative_params=False  # Apenas temp=0
)

logger.success(f"✓ Annotator inicializado com {len(annotator.models)} modelos")

01:32:56 | Cache carregado: 14710 entradas
01:32:56 | Cache LangChain ativado: ..\..\data\.cache\langchain\langchain_cache.db
01:32:56 | Template do prompt preparado
01:32:56 | LLMAnnotator inicializado
01:32:56 | Modelos: 5 | Categorias: 5
01:32:56 | ✓ Annotator inicializado com 5 modelos


## 4) Executar Anotação

In [7]:
df

,text,label,label_description
0,impostor has a handful of thrilling moments an...,1,negative sentiment
1,the acting in pauline and paulette is good all...,4,very positive sentiment
2,"mr. polanski is in his element here : alone , ...",3,positive sentiment
3,a cop story that understands the medium amazin...,3,positive sentiment
4,the most horrific movie experience i 've had s...,0,very negative sentiment
...,...,...,...
495,schindler 's list it ai n't .\n,1,negative sentiment
496,lawrence should stick to his day job .\n,1,negative sentiment
497,entertaining despite its one-joke premise with...,2,neutral sentiment
498,"while the glass slipper does n't quite fit , p...",2,neutral sentiment


In [8]:
annotator.annotation_engine.template

'You are an expert data annotator with extensive experience in text classification tasks.\n\nYour task is to classify the following text into one of the predefined categories with high precision.\n\n**Instructions:**\n1. Read the text carefully and understand its context\n2. Consider the nuances and implicit meanings\n3. Select the most appropriate category based on the content\n4. Be consistent with your classification criteria\n5. If the text is ambiguous, choose the most likely category based on dominant features\n\n**Available Categories:**\n- 0: very negative sentiment\n- 1: negative sentiment\n- 2: neutral sentiment\n- 3: positive sentiment\n- 4: very positive sentiment\n\n**Text to classify (Opinion / Sentiment):**\n{text}\n\n**Important Guidelines:**\n- Provide ONLY the category number as your response\n- Do not include explanations\n- Be objective and avoid bias\n- Consider edge cases carefully\n- Maintain consistency across similar texts\n\n**Your classification to the opinio

### - Testar anotação única

In [9]:
# Parâmetros de anotação
num_repetitions = 3
text = texts[2]
model = "llama3.1-8b"

total_annotation = len(text) * num_repetitions
logger.warning(f"  Modelo: {model}")
logger.warning(f"  Texto: {len(text)}")
logger.warning(f"  Repetições: {num_repetitions}")

annotations = annotator.annotate_single(
    text=text,
    model=model,
    num_repetitions=num_repetitions,
    use_cache=False
)

logger.success("✓ Anotação completa")

annotations

01:32:56 |   Modelo: llama3.1-8b
01:32:56 |   Texto: 182
01:32:56 |   Repetições: 3
01:33:04 | ✓ Anotação completa


[2, 2, 2]

### - Anotando dataset completo

In [10]:
output_file = annotator.results_dir / "annotations_complete.csv"
use_old_annotations = False

if use_old_annotations and output_file.exists():
    logger.warning(f"Arquivo já existe: {output_file}")
    logger.warning("Carregando anotações salvas e pulando anotação...")
    df_annotations = pd.read_csv(output_file)
    logger.success("✓ Anotações carregadas do disco")
    
else: 
    num_repetitions = 1
    df_annotations = annotator.annotate_dataset(
        texts=texts,
        num_repetitions=num_repetitions,
        use_cache=True
    )


logger.success("✓ Anotações completas")
df_annotations

01:33:04 | Iniciando anotação
01:33:04 | Textos: 500 | Modelos: 5 | Repetições: 1
01:33:04 | Total de anotações: 2500
Anotando: 100%|██████████| 500/500 [4:54:14<00:00, 35.31s/it]  
06:27:19 | Anotações completas salvas: ..\..\results\sst1\annotations_complete.csv
06:27:19 | ✓ Anotações completas


,text_id,text,deepseek-r1-8b_rep1,deepseek-r1-8b_consensus,deepseek-r1-8b_consensus_score,qwen3-8b_rep1,qwen3-8b_consensus,qwen3-8b_consensus_score,gemma3-4b_rep1,gemma3-4b_consensus,gemma3-4b_consensus_score,mistral-7b_rep1,mistral-7b_consensus,mistral-7b_consensus_score,llama3.1-8b_rep1,llama3.1-8b_consensus,llama3.1-8b_consensus_score
0,0,impostor has a handful of thrilling moments an...,1,1,1.0,1,1,1.0,1,1,1.0,1,1,1.0,1,1,1.0
1,1,the acting in pauline and paulette is good all...,3,3,1.0,3,3,1.0,3,3,1.0,3,3,1.0,2,2,1.0
2,2,"mr. polanski is in his element here : alone , ...",3,3,1.0,3,3,1.0,3,3,1.0,3,3,1.0,2,2,1.0
3,3,a cop story that understands the medium amazin...,4,4,1.0,4,4,1.0,2,2,1.0,3,3,1.0,3,3,1.0
4,4,the most horrific movie experience i 've had s...,0,0,1.0,0,0,1.0,1,1,1.0,1,1,1.0,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,schindler 's list it ai n't .\n,1,1,1.0,1,1,1.0,1,1,1.0,1,1,1.0,1,1,1.0
496,496,lawrence should stick to his day job .\n,1,1,1.0,1,1,1.0,1,1,1.0,1,1,1.0,1,1,1.0
497,497,entertaining despite its one-joke premise with...,3,3,1.0,3,3,1.0,3,3,1.0,3,3,1.0,2,2,1.0
498,498,"while the glass slipper does n't quite fit , p...",3,3,1.0,3,3,1.0,3,3,1.0,2,2,1.0,2,2,1.0


### - Metricas por modelo

In [11]:
df_annotations["ground_truth"] = ground_truth if ground_truth else None
df_metrics = annotator.evaluate_model_metrics(
    df_annotations, 
    ground_truth_col="ground_truth", 
    output_csv=True
)

06:27:19 | Calculando métricas por modelo...
06:27:19 | Total de linhas avaliadas: 500
06:27:19 | Métricas para deepseek-r1-8b: Acc=0.5180, F1=0.4994, Prec=0.5224, Rec=0.5180, Cov=1.0000
06:27:19 | Métricas para qwen3-8b: Acc=0.5300, F1=0.5137, Prec=0.5233, Rec=0.5300, Cov=1.0000
06:27:19 | Métricas para gemma3-4b: Acc=0.4380, F1=0.3473, Prec=0.3817, Rec=0.4380, Cov=1.0000
06:27:19 | Métricas para mistral-7b: Acc=0.4380, F1=0.3755, Prec=0.3769, Rec=0.4380, Cov=1.0000
06:27:19 | Métricas para llama3.1-8b: Acc=0.3820, F1=0.3329, Prec=0.5205, Rec=0.3820, Cov=0.9980
06:27:19 | Métricas por modelo salvas em: ..\..\results\sst1\model_metrics.csv
06:27:19 | ✓ Métricas calculadas com sucesso


## 5) Calcular Consenso

In [12]:
from src.llm_annotation_system.consensus.consensus_calculator import ConsensusCalculator
from src.llm_annotation_system.consensus.consensus_evaluator import ConsensusEvaluator
from src.experiments.base_experiment import EXPERIMENT_CONFIG


# Inicializar calculador
calculator = ConsensusCalculator(
    consensus_threshold=EXPERIMENT_CONFIG['consensus'].get('threshold', 0.8),
    default_strategy=EXPERIMENT_CONFIG['consensus'].get('strategy', "majority_vote")
)

analyzer = ConsensusEvaluator(categories=categories, calculator=calculator, output_dir=annotator.results_dir)

df_with_consensus = analyzer.compute_consensus(df_annotations)

06:27:19 | Executando cálculo de consenso interno...
06:27:19 | Calculando consenso...
06:27:19 | Consenso calculado:
06:27:19 |   Alto (≥80%): 293 (58.6%)
06:27:19 |   Médio (60-80%): 172 (34.4%)
06:27:19 |   Baixo (<60%): 35 (7.0%)
06:27:19 |   Problemáticos: 34 (6.8%)
06:27:19 |   Itens que precisam de revisão: 0 (0.0%)
06:27:19 | Cálculo de consenso finalizado.


### - Estatisticas

In [13]:
logger.info("\n📊 Estatísticas de Consenso:")
logger.info(f"  Média: {df_with_consensus['consensus_score'].mean():.2%}")
logger.info(f"  Mediana: {df_with_consensus['consensus_score'].median():.2%}")
logger.info(f"  Desvio padrão: {df_with_consensus['consensus_score'].std():.2%}")

06:27:19 | 
📊 Estatísticas de Consenso:
06:27:19 |   Média: 75.28%
06:27:19 |   Mediana: 80.00%
06:27:19 |   Desvio padrão: 18.09%


### - Distribuição por nível

In [14]:
# Distribuição por nível
levels = df_with_consensus['consensus_level'].value_counts()
logger.info("Distribuição por nível:")
for level, count in levels.items():
    logger.info(f"  {level}: {count} ({count/len(df_with_consensus):.1%})")

06:27:19 | Distribuição por nível:
06:27:19 |   high: 293 (58.6%)
06:27:19 |   medium: 172 (34.4%)
06:27:19 |   low: 35 (7.0%)


## - Visualizar consenso

In [15]:
from src.llm_annotation_system.consensus.consensus_visualizer import ConsensusVisualizer

visualizer = ConsensusVisualizer(output_dir=annotator.results_dir)

In [16]:
visualizer.plot_score_and_levels(
    df_with_consensus=df_with_consensus,
    levels=levels
)

✓ Gráfico salvo: score_and_levels.html


## 7️⃣ Análise Detalhada de Consenso

### - Gerando Report

In [17]:
# Gerar relatório
report = analyzer.generate_consensus_report(
    df=df_with_consensus
)

logger.success("✓ Relatório gerado")

06:27:21 | Gerando relatório completo de consenso...
06:27:21 | Fleiss' Kappa: 0.423 (Moderado)
06:27:21 | Casos problemáticos: 35
06:27:21 | Relatório de consenso gerado com sucesso.
06:27:21 | ✓ Relatório gerado


### - Pairwise_agreement

In [18]:
logger.info("\n📊 Gerando heatmap de concordância...")
visualizer.plot_agreement_heatmap(
    agreement_df=report['pairwise_agreement'],
    title='Matriz de Concordância entre Modelos',
)

06:27:21 | 
📊 Gerando heatmap de concordância...


✓ Heatmap salvo: agreement_heatmap.html


### - Cohens Kappa

In [19]:
logger.info("\n📊 Gerando heatmap de Cohens_Kappa...")
visualizer.plot_kappa_heatmap(
    kappa_df=report['cohens_kappa']
)

06:27:21 | 
📊 Gerando heatmap de Cohens_Kappa...


✓ Heatmap salvo: kappa_heatmap.html


### - Casos problemáticos

In [20]:
# Casos problemáticos
problematic = report.get('problematic_cases')
if problematic is not None and len(problematic) > 0:
    logger.warning(f"\n⚠️  {len(problematic)} casos problemáticos identificados")
    display(problematic)
else:
    logger.success("\n✓ Nenhum caso problemático identificado")

06:27:21 | 
⚠️  35 casos problemáticos identificados


,text_id,text,consensus_score,annotations,entropy
0,3,a cop story that understands the medium amazin...,0.4,"{4: 2, 2: 1, 3: 2}",1.521928
1,5,"massoud 's story is an epic , but also a trage...",0.4,"{3: 2, 2: 2, 1: 1}",1.521928
2,13,deliberately and skillfully uses ambiguity to ...,0.4,"{4: 1, 3: 2, 2: 2}",1.521928
3,27,it 's absolutely spooky how lillard channels t...,0.4,"{0: 1, 1: 2, 3: 2}",1.521928
4,29,`` one look at a girl in tight pants and big t...,0.4,"{1: 2, 0: 2, -1: 1}",1.521928
5,79,"but , like silence , it 's a movie that gets u...",0.4,"{2: 2, 1: 1, 3: 2}",1.521928
6,87,both damning and damned compelling .\n,0.4,"{2: 2, 1: 1, 3: 2}",1.521928
7,114,filmmakers david weissman and bill weber benef...,0.4,"{3: 2, 4: 1, 2: 2}",1.521928
8,128,i 'm not generally a fan of vegetables but thi...,0.4,"{3: 2, 1: 1, 2: 2}",1.521928
9,135,generic thriller junk .\n,0.4,"{0: 2, 1: 1, 2: 2}",1.521928


## 8️⃣ Validação com Ground Truth

In [21]:
if ground_truth:
    accuracy, cls_report, cm = analyzer.evaluate_ground_truth(
        df_with_consensus=df_with_consensus
    )

    visualizer.plot_confusion_matrix(
        cm=cm,
        categories=categories
    )
    
else:
    logger.info("⚠️ Ground truth não disponível – pulando validação")

06:27:21 | 
🎯 Accuracy: 45.80%
06:27:21 | 
Classification Report:


{'0': {'precision': 0.5333333333333333, 'recall': 0.12121212121212122, 'f1-score': 0.19753086419753085, 'support': 66.0}, '1': {'precision': 0.4605809128630705, 'recall': 0.8283582089552238, 'f1-score': 0.592, 'support': 134.0}, '2': {'precision': 0.3, 'recall': 0.19148936170212766, 'f1-score': 0.23376623376623376, 'support': 94.0}, '3': {'precision': 0.5, 'recall': 0.689922480620155, 'f1-score': 0.5798045602605864, 'support': 129.0}, '4': {'precision': 0.5, 'recall': 0.03896103896103896, 'f1-score': 0.07228915662650602, 'support': 77.0}, 'accuracy': 0.458, 'macro avg': {'precision': 0.45878284923928075, 'recall': 0.3739886422901333, 'f1-score': 0.3350781629701714, 'support': 500.0}, 'weighted avg': {'precision': 0.45623568464730296, 'recall': 0.458, 'f1-score': 0.38940023268983925, 'support': 500.0}}


✓ Matriz de confusão salva em: ..\..\results\sst1\graphics\confusion_matrix.html


## 9️⃣ Exportar Resultados

In [22]:
import json
from pathlib import Path

# Criar diretório
results_dir = annotator.results_dir
results_dir = results_dir.joinpath("summary")
results_dir.mkdir(parents=True, exist_ok=True)

# Salvar CSVs
df_with_consensus.to_csv(results_dir / 'dataset_anotado_completo.csv', index=False)
logger.info(f"✓ Salvos: {len(df_with_consensus)} registros")

# Alta confiança
high_conf = df_with_consensus[df_with_consensus['consensus_score'] >= 0.8]
high_conf.to_csv(results_dir / 'alta_confianca.csv', index=False)
logger.info(f"✓ Alta confiança: {len(high_conf)} registros")

# Necessita revisão
low_conf = df_with_consensus[df_with_consensus['consensus_score'] < 0.8]
low_conf.to_csv(results_dir / 'necessita_revisao.csv', index=False)
logger.info(f"✓ Necessita revisão: {len(low_conf)} registros")

# Sumário JSON
summary = {
    'dataset': {
        'name': dataset_name,
        'total_texts': len(texts),
        'categories': categories,
        'has_ground_truth': ground_truth is not None
    },
    'config': {
        'models': DEFAULT_MODELS,
        'total_models': len(annotator.models),
        'use_alternative_params': annotator.use_alternative_params,
        'num_repetitions': num_repetitions,
        'total_annotations': len(texts) * len(annotator.models) * num_repetitions
    },
    'results': {
        'consensus_mean': float(df_with_consensus['consensus_score'].mean()),
        'consensus_median': float(df_with_consensus['consensus_score'].median()),
        'high_consensus': int((df_with_consensus['consensus_level'] == 'high').sum()),
        'medium_consensus': int((df_with_consensus['consensus_level'] == 'medium').sum()),
        'low_consensus': int((df_with_consensus['consensus_level'] == 'low').sum()),
    },
    'metrics': {
        'fleiss_kappa': float(report['fleiss_kappa']),
        'fleiss_interpretation': report['fleiss_interpretation']
    }
}

if ground_truth:
    summary['validation'] = {
        'classification_report': cls_report
    }

with open(results_dir / 'sumario_experimento.json', 'w') as f:
    json.dump(summary, f, indent=2)

logger.success("\n✓ Resultados exportados com sucesso!")

06:27:21 | ✓ Salvos: 500 registros
06:27:21 | ✓ Alta confiança: 293 registros
06:27:21 | ✓ Necessita revisão: 207 registros
06:27:21 | 
✓ Resultados exportados com sucesso!


## 10) Resumo Final

In [23]:
logger.info("\n" + "="*80)
logger.success("RESUMO DO EXPERIMENTO")
logger.info("="*80)

logger.info(f"\n📊 Dataset: {dataset_name}")
logger.info(f"  Textos: {len(texts)}")
logger.info(f"  Categorias: {len(categories)}")

logger.info(f"\n🤖 Configuração:")
logger.info(f"  Modelos base: {len(DEFAULT_MODELS)}")
logger.info(f"  Total modelos: {len(annotator.models)}")
logger.info(f"  Alternative params: {annotator.use_alternative_params}")
logger.info(f"  Repetições: {num_repetitions}")

logger.info(f"\n📈 Consenso:")
logger.info(f"  Média: {df_with_consensus['consensus_score'].mean():.2%}")
logger.info(f"  Fleiss' Kappa: {report['fleiss_kappa']:.3f} ({report['fleiss_interpretation']})")

if ground_truth:
    logger.info(f"\n🎯 Validação:")
    logger.info(f"  Accuracy: {accuracy:.2%}")

logger.info(f"\n📁 Arquivos gerados em: {results_dir}/")

cache_stats = annotator.get_cache_stats()
logger.info(f"\n💾 Cache: {cache_stats['total_entries']} entradas")

logger.success("\n✅ Análise completa!")

06:27:21 | 
06:27:21 | RESUMO DO EXPERIMENTO
06:27:21 | ================================================================================
06:27:21 | 
📊 Dataset: sst1
06:27:21 |   Textos: 500
06:27:21 |   Categorias: 5
06:27:21 | 
🤖 Configuração:
06:27:21 |   Modelos base: 5
06:27:21 |   Total modelos: 5
06:27:21 |   Alternative params: False
06:27:21 |   Repetições: 1
06:27:21 | 
📈 Consenso:
06:27:21 |   Média: 75.28%
06:27:21 |   Fleiss' Kappa: 0.423 (Moderado)
06:27:21 | 
🎯 Validação:
06:27:21 |   Accuracy: 45.80%
06:27:21 | 
📁 Arquivos gerados em: ..\..\results\sst1\summary/
06:27:21 | 
💾 Cache: 17165 entradas
06:27:21 | 
✅ Análise completa!
